In [0]:
!kill -9 -1

In [1]:
!pip install kaggle

  Running setup.py bdist_wheel for kaggle ... - \ done
  Stored in directory: /content/.cache/pip/wheels/a4/29/3e/1065ec22dad7255300b56c56611abeb064300be05ae4bce5ed
Successfully built kaggle


In [0]:
from googleapiclient.discovery import build

In [3]:
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth
auth.authenticate_user()
drive_service = build('drive', 'v3')
results = drive_service.files().list(q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])
filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname( filename), exist_ok=True)
request = drive_service.files().get_media(fileId=kaggle_api_key[0] ['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


In [4]:
!kaggle competitions download -c bronchitis -p /content/kaggle

Train_Labels.csv: Downloaded 6KB of 6KB
Sample_Submission.csv: Downloaded 2KB of 2KB
Test_Images.zip: Downloaded 823MB of 823MB
Train_Images.zip: Downloaded 3GB of 3GB


In [5]:
cd kaggle

/content/kaggle


In [6]:
ls

Sample_Submission.csv  Test_Images.zip  Train_Images.zip  Train_Labels.csv


In [0]:
import zipfile

zip_ref = zipfile.ZipFile('Test_Images.zip', 'r')
zip_ref.extractall()
zip_ref.close()

In [0]:
import zipfile

zip_ref = zipfile.ZipFile('Train_Images.zip', 'r')
zip_ref.extractall()
zip_ref.close()

In [9]:
ls

Sample_Submission.csv  Test_Images.zip  Train_Images.zip
Test_Images/           Train_Images/    Train_Labels.csv


In [11]:
import pandas as pd
df=pd.read_csv('Train_Labels.csv')
df

,Images,Labels
0,CR_0258,0
1,CR_0467,1
2,CR_0627,1
3,CR_0516,1
4,CR_0496,1
5,CR_0415,1
6,CR_0257,0
7,CR_0255,0
8,CR_0125,0
9,CR_0176,0


In [0]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
import torch.nn as nn


class MnistCNNModel(nn.Module):
    def __init__(self):
        super(MnistCNNModel, self).__init__()
        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=0)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=4)
        self.fc1 = nn.Linear(61504, 10)

    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)
        out = self.maxpool1(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        return out

In [0]:
from PIL import Image

def register_extension(id, extension):
    Image.EXTENSION[extension.lower()] = id.upper()
Image.register_extension = register_extension
def register_extensions(id, extensions):
    for extension in extensions:
        register_extension(id, extension)
Image.register_extensions = register_extensions

In [0]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import transforms
from torch.utils.data.dataset import Dataset  # For custom datasets

class CustomDatasetFromImages(Dataset):
    def __init__(self, csv_path,img_path):
        """
        Args:
            csv_path (string): path to csv file
            img_path (string): path to the folder where images are
            transform: pytorch transforms for transforms and tensor conversion
        """
        # Transforms
        self.to_tensor = transforms.Compose([transforms.Resize((255,255)),transforms.ToTensor()])
        # Read the csv file
        self.data_info = pd.read_csv(csv_path)
        # First column contains the image paths
        self.image_arr = np.asarray(img_path + self.data_info.iloc[:, 0]+'.png')
        # Second column is the labels
        self.label_arr = np.asarray(self.data_info.iloc[:, 1])
        # Calculate len
        self.data_len = len(self.data_info.index)

    def __getitem__(self, index):
        # Get image name from the pandas df
        single_image_name = self.image_arr[index]
        # Open image
        img_as_img = Image.open(single_image_name).convert('1')

        # Check if there is an operation
        img_as_tensor = self.to_tensor(img_as_img)

        # Get label(class) of the image based on the cropped pandas column
        single_image_label = self.label_arr[index]

        return (img_as_tensor, single_image_label)

    def __len__(self):
        return self.data_len




In [0]:
    transformations = transforms.Compose([transforms.ToTensor()])
    
    custom_mnist_from_images =  \
        CustomDatasetFromImages('Train_Labels.csv','Train_Images/')

    mn_dataset_loader = torch.utils.data.DataLoader(dataset=custom_mnist_from_images,
                                                    batch_size=10,
                                                    shuffle=False)

    model = MnistCNNModel()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

    for i, (images, labels) in enumerate(mn_dataset_loader):
        images = Variable(images)
        labels = Variable(labels)
        # Clear gradients
        optimizer.zero_grad()
        # Forward pass
        outputs = model(images)
        # Calculate loss
        loss = criterion(outputs, labels)
        # Backward pass
        loss.backward()
        # Update weights
        optimizer.step()
        break

In [0]:
import string
   
csv = open("Test_Labels1.csv", "w") 
columnTitleRow = "Images, Labels\n"
csv.write(columnTitleRow)
for filename in os.listdir('Test_Images/'):
   csv.write(str.replace(filename,'.png','')+","+"0"+"\n")


In [23]:
df=pd.read_csv("Train_Labels.csv")
df[:1]

,Images,Labels
0,CR_0258,0


In [26]:
 df_t=pd.read_csv("Test_Labels1.csv")
 df_t[:1]

,Images,Labels
0,CR_0068,0


In [33]:
    transformations = transforms.Compose([transforms.ToTensor()])
    
    custom_mnist_from_images_t =  \
        CustomDatasetFromImages('Test_Labels1.csv','Test_Images/')

    mn_dataset_loader_t = torch.utils.data.DataLoader(dataset=custom_mnist_from_images_t,
                                                    batch_size=1,
                                                    shuffle=False)
    
    model.eval()
    for i, (images, labels) in enumerate(mn_dataset_loader_t):
        images = Variable(images)
        labels = Variable(labels)
        output = model(images)
        pred = output.data.max(1)[1] # get the index of the max log-probability
        print(str(i)+","+str(pred[0]))
       
        

0,1
1,1
2,1
3,1
4,1
5,1
6,1
7,1
8,1
9,1
10,1
11,1
12,1
13,1
14,1
15,1
16,1
17,1
18,1
19,1
20,1
21,1
22,1
23,1
24,1
25,1
26,1
27,1
28,1
29,1
30,1
31,1
32,1
33,1
34,1
35,1
36,1
37,1
38,1
39,1
40,1
41,1
42,1
43,1
44,1
45,1
46,1
47,1
48,1
49,1
50,1
51,1
52,1
53,1
54,1
55,1
56,1
57,1
58,1
59,1
60,1
61,1
62,1
63,1
64,1
65,1
66,1
67,1
68,1
69,1
70,1
71,1
72,1
73,1
74,1
75,1
76,1
77,1
78,1
79,1
80,1
81,1
82,1
83,1
84,1
85,1
86,1
87,1
88,1
89,1
90,1
91,1
92,1
93,1
94,1
95,1
96,1
97,1
98,1
99,1
100,1
101,1
102,1
103,1
104,1
105,1
106,1
107,1
108,1
109,1
110,1
111,1
112,1
113,1
114,1
115,1
116,1
117,1
118,1
119,1
120,1
121,1
122,1
123,1
124,1
125,1
126,1
127,1
128,1
129,1
130,1
131,1
132,1
133,1
134,1
135,1
136,1
137,1
138,1
139,1
140,1
141,1
142,1
143,1
144,1
145,1
146,1
147,1
148,1
149,1
150,1
151,1
152,1
153,1
154,1
155,1
156,1
157,1
158,1
159,1
